In [2]:
#/usr/bin/env python
"""
Linux如何声明环境变量
三种：临时 全局 全用户
export JAVA_HOME=/usr/lib/jvm/java-11-openjdk
export PATH=/usr/local/bin:$PATH

vim ~/.bashrc

vim /etc/profile
export JAVA=

#? 有问题
"""

#1 加载模型 pipeline
"""
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)
print(generator)
"""
#2模型内部是如何运行的？
#token+model+head_model+pretraining

# myself
#token,model,head_model,
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch

checkpoint="distilbert-base-uncased-finetuned-sst-2-english"
token=AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs=token(raw_inputs,padding=True,truncation=True,return_tensors="pt") #长度填充，位置填充
#print(inputs)

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model=AutoModelForSequenceClassification.from_pretrained(checkpoint)
out=model(**inputs)
print(out.logits)

pred=torch.nn.functional.softmax(out.logits,dim=-1)
print(pred)

#model
#模型内部是如何组成的？
from transformers import BertConfig, BertModel
contig=BertConfig()
#model=BertModel(contig)#只加载配置模型
model=BertModel.from_pretrained("bert-ase-cased")#经过预训练

model.save_pretrained("../model/bert")

# 标记器

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)
tensor([[4.0195e-02, 9.5981e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


OSError: bert-ase-cased is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [ ]:
#2.4 标记器
'''
model input 是矩阵
token 把输入数据 转化为 矩阵
三种：基于 单词 字母 子词subword
'''
#加载tokenizer并保存到本地


'\nmodel input 是矩阵\ntoken 把输入数据 转化为 矩阵\n三种：基于 单词 字母 子词subword\n'